# Algoithmic Fairness, Accountability and Ethics
## Assignment 2 (Template)

In [ ]:
from folktables import ACSDataSource, BasicProblem, generate_categories
import numpy as np
import scipy.optimize as opt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats.contingency import association
from scipy.stats import pointbiserialr

### 1. Load and Preprocess the data
We are going to work with the [Folktables](https://github.com/socialfoundations/folktables#quick-start-examples) dataset (*you have already worked with it*).

1. As last week, we are still predicting the *Total person's income*  (I've digitized  it in  `target_transform=lambda x: x > 25000`).
2. Today, we are going to implement two methods for data debiasing: [Fair PCA](https://deepai.org/publication/efficient-fair-pca-for-fair-representation-learning) and [A Geometric Solution to Fair Representations](https://dl.acm.org/doi/10.1145/3375627.3375864).
3. We are going to evaluate the performance on two sensitive features: `SEX` (i.e. *Males* and *Females*) and `RAC1P` (we will consider only *Whites* and *African-Americans*)
4. I updated the filtering method `adult_filter` to keep the specified groups.

In [ ]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)

def adult_filter(data):
    """Mimic the filters in place for Adult data.
    Adult documentation notes: Extraction was done by Barry Becker from
    the 1994 Census database. A set of reasonably clean records was extracted
    using the following conditions:
    ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
    """
    df = data
    df = df[df['AGEP'] > 16]
    df = df[df['PINCP'] > 100]
    df = df[df['WKHP'] > 0]
    df = df[df['PWGTP'] >= 1]
    df = df[df["RAC1P"] < 3] ## keep only Whites and African-Americans
    return df


ACSIncomeNew = BasicProblem(
    features=[
        'AGEP', # Age
        'COW', # Class of worker i.e private for profit, gov, non-profit
        'SCHL', # Education
        'MAR', # marriage status
        'CIT', # citizenship status
        'RELP', # relationship to reference person
        'WKHP', # hours worked per week
        'PWGTP', # weight
        'SEX', # 1. male 2. female
        'RAC1P'# 1. white, 2 black
    ],
    target='PINCP',
    target_transform=lambda x: x > 25000,    
    group=['SEX', "RAC1P"],
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

In [ ]:
features, target, protected = ACSIncomeNew.df_to_pandas(acs_data)


In [ ]:
features

In [ ]:
# Age and Sex
# association(np.array(features[["AGEP","SEX"]], dtype=int))
# Sex and age
print("Sex and Age",pointbiserialr(features["SEX"]-1, features["AGEP"]))
# Sex and school
print("Sex and school",pointbiserialr(features["SEX"]-1, features["SCHL"]))
# Sex and marriage status
print("Sex and marriage status cramerv",round(association(np.array(features[["SEX","MAR"]], dtype=int)),3))
# Sex and Work class
print("Sex and work class cramerv",round(association(np.array(features[["SEX","COW"]], dtype=int)),3))
# Sex and citizenship status
print("Sex and citizen status cramerv",round(association(np.array(features[["SEX","CIT"]], dtype=int)),3))
# Sex and hours worked per week
print("Sex and Work Hours",pointbiserialr(features["SEX"]-1, features["WKHP"]))
# Sex and weight
print("Sex and Weight",pointbiserialr(features["SEX"]-1, features["PWGTP"]))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features, target, protected, test_size=0.3, random_state=0, shuffle=True)

In [ ]:
N = 500

X_train = X_train[:N]
y_train = y_train[:N]
group_train = group_train[:N]
X_test = X_test[:N]
y_test = y_test[:N]
group_test = group_test[:N]

## Task 1

In [ ]:
import seaborn as sns

for cat in ['SEX', 'RAC1P']:
    for i in range(1,3):
        print(f"{cat} {i} count:",len(group_train[group_train[cat] == i]))



In [ ]:
males = group_train[group_train['SEX'] == 1]
females = group_train[group_train['SEX'] == 2]

for i in range(1,3):
    print(f"Male race {i}", len(males[males['RAC1P'] == i]))
    print(f"Female race {i}", len(females[females['RAC1P'] == i]))

### Bias in Training Data
We find around 10% more men than women in our training data, and more than 11 times more Whites than African-Americans (!)

## 1.2 Proxies

Looking at the distribution of features for each gender, a model could potentially use low weight < ~50 as a proxy variable for woman and a high amount of working hours as a proxy for male.

In [ ]:
sns.histplot(data=X_train.assign(PINCP=y_train), x='PINCP', hue='SEX', multiple="dodge", shrink=0.8)

In [ ]:
for feature in X_train.drop(['SEX'], axis=1).columns:
    sns.histplot(data=X_train, x=feature, hue='SEX', multiple="dodge", shrink=0.8)
    plt.show()

## Task 2

In [ ]:
definition_df = data_source.get_definitions(download=True)
categories = generate_categories(features=ACSIncomeNew.features, definition_df=definition_df)
features, labels, groups = ACSIncomeNew.df_to_pandas(acs_data, categories=categories, dummies=True)
### groups now contain information about SEX and RAC1P
features.head()

In [ ]:
# Drop the "redundant" columns
features = features.drop(["RELP_Unmarried partner",
                          "CIT_U.S. citizen by naturalization",
                          "SEX_Male",
                          "SCHL_1 or more years of college credit, no degree",  
                          "MAR_Divorced", 
                          "RELP_Adopted son or daughter",
                          'COW_Working without pay in family business or farm', 
                          "RAC1P_White alone" ], axis = 1) 

print("Columns with the protected features:")
for i, f in enumerate(features.columns):
    if ("RAC1P" in f) or ("SEX" in f):
        print("Column ID: %s" %i, "(%s)"%f)

In [ ]:
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features.values, labels.values.reshape(-1), groups, test_size=0.3, random_state=0, shuffle=True)

N = 500 ### I am subsampling because it is slow on my machine

X_train = X_train[:N]
y_train = y_train[:N]
group_train = group_train[:N]
X_test = X_test[:N]
y_test = y_test[:N]
group_test = group_test[:N]

In [ ]:
############
# YOUR CODE
############

#### Task 2.2.
Use the following arguments in the `opt.fmin_funct`: `xtol=1e-4, ftol=1e-4,  maxfun=1000`

In [ ]:
gammas = np.logspace(1e-5,1e-2,10)
###########
# YOUR CODE
###########

#### Task 2.3
Use the following arguments in the `opt.fmin_funct`: ` xtol=1e-3, ftol=1e-3, approx_grad=True, maxfun=1000`

In [ ]:
lambdas = np.array([1e-3, 5e-3, 1e-2, 5e-2, 0.1, 1])
###########
# YOUR CODE
###########